In [84]:
import pandas as pd
import numpy as np
seed = 42
import category_encoders as ce

## Obtener los datos del "Credit Approval Dataset"

In [85]:
ruta = "/home/jair/Escritorio/Tec. Av. Patrones/Tareas/Tarea1/datasets/crx.csv"
df = pd.read_csv(ruta, na_values = '?')
df.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,Class
0,b,3083.0,0.0,u,g,w,v,125.0,t,t,1,f,g,202.0,0,positive
1,a,5867.0,446.0,u,g,q,h,304.0,t,t,6,f,g,43.0,560,positive
2,a,245.0,5.0,u,g,q,h,15.0,t,f,0,f,g,280.0,824,positive
3,b,2783.0,154.0,u,g,w,v,375.0,t,t,5,t,g,100.0,3,positive
4,b,2017.0,5625.0,u,g,w,v,171.0,t,f,0,f,s,120.0,0,positive


In [86]:
df.shape
# 690 vectores
# 15 características

(690, 16)

## Limpieza de repetidos e indiscernibles  

In [87]:
# nulos
df = df.dropna()

# repetidos
df = df.drop_duplicates(keep = 'first')

# indiscernibles
características = df.drop(' Class', axis = 1)
df = df[~df.duplicated(subset = características, keep = False)]

In [88]:
df.shape
# 653 vectores
# 15 características
# Se eliminaron 59 vectores

(653, 16)

In [89]:
# Tienen un espacio los nombres de las oclumnas
df.columns = df.columns.str.strip()
df.columns

Index(['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11',
       'A12', 'A13', 'A14', 'A15', 'Class'],
      dtype='object')

In [90]:
df.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,Class
0,b,3083.0,0.0,u,g,w,v,125.0,t,t,1,f,g,202.0,0,positive
1,a,5867.0,446.0,u,g,q,h,304.0,t,t,6,f,g,43.0,560,positive
2,a,245.0,5.0,u,g,q,h,15.0,t,f,0,f,g,280.0,824,positive
3,b,2783.0,154.0,u,g,w,v,375.0,t,t,5,t,g,100.0,3,positive
4,b,2017.0,5625.0,u,g,w,v,171.0,t,f,0,f,s,120.0,0,positive


In [91]:
# Etiqueta de clase numérica con un diccionario
df['Class'] = df['Class'].replace({'positive': 1, 'negative':0})
df.head()

/tmp/ipykernel_10488/501916219.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Class'] = df['Class'].replace({'positive': 1, 'negative':0})


,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,Class
0,b,3083.0,0.0,u,g,w,v,125.0,t,t,1,f,g,202.0,0,1
1,a,5867.0,446.0,u,g,q,h,304.0,t,t,6,f,g,43.0,560,1
2,a,245.0,5.0,u,g,q,h,15.0,t,f,0,f,g,280.0,824,1
3,b,2783.0,154.0,u,g,w,v,375.0,t,t,5,t,g,100.0,3,1
4,b,2017.0,5625.0,u,g,w,v,171.0,t,f,0,f,s,120.0,0,1


## Separar numéricas y categóricas

In [92]:
numericos = ['A2','A3','A8','A11','A14','A15']
numericos

['A2', 'A3', 'A8', 'A11', 'A14', 'A15']

In [93]:
categoricos = [col for col in df.columns if col not in numericos and col != 'Class']
categoricos

['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10', 'A12', 'A13']

## Corregir formato de numéricos y categóricos

In [94]:
df[numericos] = df[numericos].astype(float)
df[categoricos] = df[categoricos].astype(str)
df.dtypes

A1        object
A2       float64
A3       float64
A4        object
A5        object
A6        object
A7        object
A8       float64
A9        object
A10       object
A11      float64
A12       object
A13       object
A14      float64
A15      float64
Class      int64
dtype: object

## Pasar las características Categoricas a Numéricas

In [95]:
# Aplicar Bynary encoding a el df

encoder = ce.BinaryEncoder(cols = categoricos)
df_encoded = encoder.fit_transform(df[categoricos])

df = df.drop(columns = categoricos)
df = pd.concat([df_encoded, df], axis=1)

df.head()

,A1_0,A1_1,A4_0,A4_1,A5_0,A5_1,A6_0,A6_1,A6_2,A6_3,...,A12_1,A13_0,A13_1,A2,A3,A8,A11,A14,A15,Class
0,0,1,0,1,0,1,0,0,0,1,...,1,0,1,3083.0,0.0,125.0,1.0,202.0,0.0,1
1,1,0,0,1,0,1,0,0,1,0,...,1,0,1,5867.0,446.0,304.0,6.0,43.0,560.0,1
2,1,0,0,1,0,1,0,0,1,0,...,1,0,1,245.0,5.0,15.0,0.0,280.0,824.0,1
3,0,1,0,1,0,1,0,0,0,1,...,0,0,1,2783.0,154.0,375.0,5.0,100.0,3.0,1
4,0,1,0,1,0,1,0,0,0,1,...,1,1,0,2017.0,5625.0,171.0,0.0,120.0,0.0,1


In [ ]:
df.shape
# se paso de 15 a 29 características por la codificación binaria pero es lo que hay

(653, 29)